Extracting the Data from Spam Assasin

In [1]:
# prompt: unzip tar.bz file
import os
import urllib.request
import tarfile

urllib.request.urlretrieve("https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2", "ham.tar.bz2")
urllib.request.urlretrieve("https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2", "spam.tar.bz2")

# Extract the contents of the tar.bz2 file
with tarfile.open("ham.tar.bz2", "r:bz2") as tar:
    tar.extractall()

with tarfile.open("spam.tar.bz2", "r:bz2") as tar:
    tar.extractall()

os.rename('easy_ham', 'ham')

In [ ]:

os.listdir(os.getcwd())

['.config',
 '.ipynb_checkpoints',
 'ham',
 'spam.tar.bz2',
 'spam',
 'ham.tar.bz2',
 'sample_data']

In [43]:
import os
import pandas as pd
def load_emails(directory):
    emails = []
    labels = []
    for label in ['spam', 'ham']:
        folder = os.path.join(directory, label)
        for filename in os.listdir(folder):
            with open(os.path.join(folder, filename), 'r', encoding='latin1') as file:
                emails.append(file.read())
                labels.append(label)
    return emails, labels

emails,labels = load_emails('/content/')
df = pd.DataFrame({'email': emails, 'label': labels})

In [44]:
df

,email,label
0,From info@qpas.co.uk Mon Sep 2 12:30:08 2002...,spam
1,Received: from qrq.cc.ntu.edu.tw (giga.tw.free...,spam
2,From cheapsmokes@terra.es Wed Sep 11 13:57:36...,spam
3,From powerballinfo@powerballinfo.cjb.net Thu ...,spam
4,From happy6251222@yahoo.com Mon Aug 26 15:13:...,spam
...,...,...
3047,From rssfeeds@jmason.org Sun Oct 6 22:54:34 ...,ham
3048,From rssfeeds@jmason.org Tue Oct 8 10:55:36 ...,ham
3049,From fork-admin@xent.com Thu Sep 5 11:31:06 ...,ham
3050,From ilug-admin@linux.ie Mon Oct 7 12:06:33 ...,ham


In [37]:
import nltk
from nltk.corpus import stopwords

In [39]:
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
import re
def preprocess_text(text):
    # Remove special characters and numbers
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

In [49]:
df['cleaned_email'] = df['email'].apply(preprocess_text)

In [53]:
df.iloc[0]["cleaned_email"]

'info qpas co uk mon sep return path info qpas co uk delivered zzzz localhost example com received localhost localhost phobos labs example com postfix esmtp id f zzzz localhost mon sep edt received phobos localhost imap fetchmail zzzz localhost single drop mon sep ist received webnote net mail webnote net dogma slashnull org esmtp id g ul uz zzzz risks jmason org fri aug received tinycomp stylezice public seve cust walt broadband ntl com webnote net smtp id waa zzzz risks jmason org fri aug message id waa webnote net support engineers info qpas co uk zzzz risks jmason org subject uk leading pc specialist mime version content type text plain charset iso date sat jun nationwide pc repairs upgrades call charges hourly charges picked home returned safely anywhere uk diagnosed repaired certified engineer meet beat price information click link http www qpas co uk pcclinic htm'

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # Use top 5000 features
X = vectorizer.fit_transform(df['cleaned_email']).toarray()
y = df['label'].apply(lambda x: 1 if x == 'spam' else 0)  # Convert labels to binary

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [61]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [62]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9869067103109657
Confusion Matrix:
 [[495   2]
 [  6 108]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       497
           1       0.98      0.95      0.96       114

    accuracy                           0.99       611
   macro avg       0.98      0.97      0.98       611
weighted avg       0.99      0.99      0.99       611



Need to use harder spam to check
